## Read first 10000 rows from json file

In [1]:
import json
import pandas as pd
def read_json_nrows(nrows,filename):
    n= 0
    with open(filename) as f:
        while n < nrows:
            if n == 0:
                line = f.readline()
                line = json.loads(line.rstrip())
                train = pd.DataFrame(line,index = [0])
            else:
                temp = pd.DataFrame(json.loads(f.readline().rstrip()),index = [n])
                train = train.append(temp)
            n = n+1
    return train

In [2]:
train_10000 = read_json_nrows(10000,'review_train.json')

In [3]:
train_10000.head(5)

,business_id,stars,text,date
0,31292,1.0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,35344,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,152538,5.0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,71871,5.0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,64913,1.0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


## Cleaning data

In [4]:
from collections import Counter
from nltk.util import ngrams
import re
import numpy as np 
from nltk.corpus import words
from nltk.corpus import brown
from nltk.corpus import wordnet
from nltk import wordpunct_tokenize

### Languages

When I first tried to apply the *detect* function I got a 'No features in text' error. So I have to find out which review is empty.

In [5]:
train_10000.loc[6687]

business_id                 137207
stars                            1
text                            :(
date           2017-07-18 20:31:03
Name: 6687, dtype: object

This review is not language, but an emoticon.

In [6]:
def not_language(text):
    # First delete all common emoticons.
    text = re.sub('(?::|;|=)(?:-)?(?:\)|\(|D|P)','',text)
    if re.sub('[\W]+','',text) == '':
        return True
    else:
        return False

For simplicity, just consider emoticons as English.

In [7]:
not_lang = train_10000[train_10000.text.apply(not_language)].index.values

In [8]:
train_10000.loc[not_lang,'lang_type'] = 'english'

from nltk.corpus import stopwords
from nltk import wordpunct_tokenize
languages_ratios = {}
for i in range(10000):
    tokens = wordpunct_tokenize(train_10000.text[i])
    words = [word.lower() for word in tokens]
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)
        languages_ratios[language] = len(common_elements)
    most_rated_language = max(languages_ratios, key=languages_ratios.get)
    train_10000.loc[i,'lang_type'] = most_rated_language

In [9]:
from langdetect import detect
for i in range(10000):
    if i in not_lang:
        continue
    else:
        train_10000.loc[i,'lang_type'] = detect(train_10000.text[i])

In [10]:
re.findall(':[\W]{0,1}',':(nmsl')

[':(']

In [11]:
train_10000.lang_type.value_counts()

en         9956
fr           37
es            3
it            1
english       1
ja            1
de            1
Name: lang_type, dtype: int64

Focus on English only at present.

In [12]:
train_10000_eng = train_10000[train_10000.lang_type == 'en']

In [46]:
train_10000_en = pd.DataFrame(train_10000_eng)

Most common words and phrases.

In [13]:
text = ' '.join(train_10000_eng.text.values)
text_trigrams = [i for i in ngrams(text.split(), 1)]
Counter(text_trigrams).most_common(20)

[(('the',), 43484),
 (('and',), 36445),
 (('I',), 27219),
 (('a',), 26196),
 (('to',), 25807),
 (('was',), 18766),
 (('of',), 15249),
 (('is',), 12814),
 (('for',), 12221),
 (('in',), 11026),
 (('it',), 9188),
 (('The',), 9180),
 (('with',), 8640),
 (('my',), 8424),
 (('that',), 8146),
 (('but',), 7072),
 (('on',), 7003),
 (('have',), 6583),
 (('you',), 6540),
 (('this',), 6388)]

In [14]:
text = ' '.join(train_10000_eng.text.values)
text_trigrams = [i for i in ngrams(text.split(), 2)]
Counter(text_trigrams).most_common(20)

[(('of', 'the'), 3443),
 (('and', 'the'), 2772),
 (('in', 'the'), 2722),
 (('it', 'was'), 2336),
 (('I', 'was'), 2307),
 (('on', 'the'), 2107),
 (('and', 'I'), 2017),
 (('to', 'the'), 1836),
 (('for', 'the'), 1732),
 (('for', 'a'), 1732),
 (('I', 'had'), 1559),
 (('I', 'have'), 1490),
 (('is', 'a'), 1378),
 (('to', 'be'), 1358),
 (('was', 'a'), 1313),
 (('this', 'place'), 1278),
 (('with', 'the'), 1262),
 (('at', 'the'), 1245),
 (('with', 'a'), 1216),
 (('to', 'get'), 1206)]

In [15]:
text = ' '.join(train_10000_eng.text.values)
text_trigrams = [i for i in ngrams(text.split(), 3)]
Counter(text_trigrams).most_common(20)

[(('and', 'it', 'was'), 516),
 (('one', 'of', 'the'), 451),
 (('a', 'lot', 'of'), 392),
 (('I', 'had', 'the'), 338),
 (('This', 'place', 'is'), 316),
 (('I', 'have', 'been'), 265),
 (('I', 'had', 'to'), 262),
 (('the', 'food', 'was'), 254),
 (('I', 'ordered', 'the'), 247),
 (('it', 'was', 'a'), 244),
 (('of', 'the', 'best'), 239),
 (('The', 'food', 'was'), 237),
 (('I', 'had', 'a'), 236),
 (('some', 'of', 'the'), 206),
 (('The', 'food', 'is'), 199),
 (('this', 'place', 'is'), 198),
 (('The', 'service', 'was'), 187),
 (('the', 'food', 'is'), 187),
 (('to', 'get', 'a'), 182),
 (('This', 'is', 'a'), 181)]

In [16]:
text = ' '.join(train_10000_eng.text.values)
text_trigrams = [i for i in ngrams(text.split(), 4)]
Counter(text_trigrams).most_common(20)

[(('one', 'of', 'the', 'best'), 126),
 (('My', 'husband', 'and', 'I'), 88),
 (('I', 'will', 'definitely', 'be'), 77),
 (('is', 'one', 'of', 'the'), 73),
 (('the', 'end', 'of', 'the'), 67),
 (('in', 'the', 'middle', 'of'), 65),
 (('for', 'the', 'first', 'time'), 65),
 (('a', 'great', 'place', 'to'), 62),
 (('some', 'of', 'the', 'best'), 60),
 (('and', 'the', 'service', 'was'), 56),
 (('you', 'are', 'looking', 'for'), 55),
 (('was', 'one', 'of', 'the'), 54),
 (('and', 'the', 'food', 'was'), 53),
 (('I', 'have', 'to', 'say'), 52),
 (('My', 'wife', 'and', 'I'), 50),
 (('and', 'the', 'food', 'is'), 50),
 (('was', 'my', 'first', 'time'), 49),
 (('one', 'of', 'my', 'favorite'), 49),
 (('the', 'rest', 'of', 'the'), 48),
 (('I', "can't", 'wait', 'to'), 47)]

Oddest words

In [17]:
text = ' '.join(train_10000_eng.text.values)
text_trigrams = [i for i in ngrams(text.split(), 1)]
Counter(text_trigrams).most_common()[-500:]

[(('qualified,',), 1),
 (('exuded',), 1),
 (("'cool'",), 1),
 (('Danny.',), 1),
 (('accord',), 1),
 (('b-day.',), 1),
 (('Terroni.',), 1),
 (('raptor',), 1),
 (('flats',), 1),
 (('gripe...no',), 1),
 (('GPS?',), 1),
 (('yardage',), 1),
 (('caliber,',), 1),
 (('troon',), 1),
 (('Quintero',), 1),
 (('parking....my',), 1),
 (('pefect',), 1),
 (('Dewy',), 1),
 (('appoinment,',), 1),
 (('in...)',), 1),
 (('HVAC/Electrical/etc...',), 1),
 (('electrical....(I',), 1),
 (('NHW',), 1),
 (('compressor',), 1),
 (('verified',), 1),
 (('TUESDAY',), 1),
 (('compressor....',), 1),
 (('occasions....',), 1),
 (('SH!*',), 1),
 (('Moe',), 1),
 (('matted',), 1),
 (('Choosing',), 1),
 (('changing,',), 1),
 (('eats!',), 1),
 (('loong',), 1),
 (('lo-mean,',), 1),
 (('Discovered',), 1),
 (('Beach.',), 1),
 (('Thrilled',), 1),
 (('favs:',), 1),
 (('DELISH!)',), 1),
 (('is..',), 1),
 (('Woo',), 1),
 (('Che.',), 1),
 (('grilles,',), 1),
 (('bugogi',), 1),
 (('thicker,',), 1),
 (('Louie,',), 1),
 (('hmmm,',), 1),


Here I found two sentences in Chinese:
- (('不要在这里吃！',), 1),
- (('我们刚在这里吃午饭，在我们的汤里发现了一个蟑螂，我把它展示给服务员，她说：哦，对不起。你不必付饭费"，这意味着他们的厨房里有很多蟑螂，他们知道，她一点都不惊讶。',),  

Most of these rare words have some punctuations.

In [18]:
text = ' '.join(train_10000_eng.text.values)
text_trigrams = [i for i in ngrams(text.split(), 2)]
Counter(text_trigrams).most_common()[-20:]

[(('visit!', "Haven't"), 1),
 (('years', 'bc'), 1),
 (('something', 'drew'), 1),
 (('whites', 'mixed'), 1),
 (('with', 'bellpeppers,'), 1),
 (('bellpeppers,', 'onions'), 1),
 (('tomatoes,', 'avocado'), 1),
 (('fruit,', 'with'), 1),
 (('muffin.', 'Super'), 1),
 (('Super', 'healthy'), 1),
 (('with', 'Pam'), 1),
 (('Pam', 'instead'), 1),
 (('they', 'willingly'), 1),
 (('willingly', 'accommodated!'), 1),
 (('accommodated!', 'This'), 1),
 (('server', '(Matt!)'), 1),
 (('(Matt!)', 'was'), 1),
 (('AWESOME!', 'Come'), 1),
 (('and', 'visit--worth'), 1),
 (('visit--worth', 'it!'), 1)]

### Check bad words

#### First emoticons

In [19]:
def find_emoticons(text):
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    if emoticons == []:
        return False
    else:
        return True

In [20]:
eg1 = np.where(train_10000_eng.text.apply(find_emoticons) == True)[0][0]

In [21]:
train_10000_eng.iloc[eg1].text

"Man, I love Toronto! Hiding in a strip mall on Overlea, find a dingy looking restaurant that serves up fantastic, cheap kabob...be warned regardless of when you come here you'll likely be waiting for a table as this place is always overrun with people...If you're a fan of perfectly grilled spicy meat, you must try this place out...now that I've been here I need to limit how many times I go here per month :)\n\nTake the drive out and have some kabob...if you don't own a car, borrow one or get a zipcar membership...totally worth it!"

Here we can find one  :)  together with two '\n' 

In [22]:
re.sub('\\n','',train_10000_eng.iloc[eg1].text)

"Man, I love Toronto! Hiding in a strip mall on Overlea, find a dingy looking restaurant that serves up fantastic, cheap kabob...be warned regardless of when you come here you'll likely be waiting for a table as this place is always overrun with people...If you're a fan of perfectly grilled spicy meat, you must try this place out...now that I've been here I need to limit how many times I go here per month :)Take the drive out and have some kabob...if you don't own a car, borrow one or get a zipcar membership...totally worth it!"

#### Check typos

In [22]:
# Most packages can't handle some words like 'nooooo'. 
def check_same(word):
    intervals = {}
    for index,letter in enumerate(word):
        if letter == word[index-1]:
            if letter in intervals.keys():
                interval = intervals.pop(letter)
                lastguy = interval[len(interval)-1] 
                if lastguy[1] == (index-1):
                    lastguy = (lastguy[0],index)
                    interval[len(interval)-1] = lastguy
                    intervals[letter] = interval
                else:
                    lastguy1 = (index-1,index)
                    interval.append(lastguy1)
                    intervals[letter] = interval
            else:
                intervals[letter] = [(index-1,index)]
    return intervals

In [23]:
# I have never seen a word with three continuous same letter, so I will delete till two.
def no_more_than_2(word,dupli):
    for key in dupli.keys():
        for interval in dupli[key]:
            length = interval[1]-interval[0]+1
            regex = '(%s'%key + '{%i})'%length
            word = re.sub(regex,key+key,word)
    return word

In [24]:
from pattern.en import suggest
from itertools import combinations
def right_spelling(word,dupli):
    key_list = []
    for key in dupli.keys():
        key_list.append(key)
    n = len(key_list)
    for i in range(n):
        for comb in combinations(key_list,i):
            for letter in comb:
                regex = '(%s'%letter + '{2})'
                new_word = re.sub(regex,letter,word)
                if new_word in brown.words():
                    return new_word
    return suggest(word)[0][0]

In [25]:
def no_typo(word):
    if len(word) == len(set(word)):
        return word
    if suggest(word)[0][1] == 1:
        return suggest(word)[0][0]
    else:
        duplicates = check_same(word)
        two = no_more_than_2(word,duplicates)
        suggest_two = suggest(two)
        if suggest_two[0][1] == 1:
            return suggest_two[0][0]
        else:
            return right_spelling(two,duplicates)
        
        return right_spelling(word)

In [26]:
no_typo('finaaallly')

'finally'

#### Lemmatization

In [27]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet

In [28]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [29]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
wnl = WordNetLemmatizer()
def lemmatizer(text):
    tokens = word_tokenize(text)
    lemmas = []
    tagged = pos_tag(tokens)
    for tag in tagged:
        wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
        lemmas.append(wnl.lemmatize(tag[0], pos=wordnet_pos))
    return lemmas

#### Stop-words

I think words which mean negative are important.

In [30]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jywsi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jywsi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jywsi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [31]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop.pop(stop.index('but'))
stop.pop(stop.index('not'))


'not'

#### Convert n't to not

In [32]:
def no_abbreviation(text):
    text = re.sub('n\'t',' not',text)
    return text

#### Adversatives

In [33]:
but = ['yet','however','nonetheless','whereas','nevertheless']
although = ['although','though','notwithstanding','albeit']

In [34]:
def change_but(text):
    for x in but:
        text = re.sub(x,'but',text)
    return text
def change_although(text):
    for x in although:
        text = re.sub(x,'although',text)
    return text
def change_adversatives(text):
    text = change_but(text)
    text = change_although(text)
    return text

Now I want to capture the key information near but and although.

def although_phrase(text):
    words = text.split()
    for (index,word) in enumerate(words):
        if word == 'altough.':
            for x in range(index,index-10,-1):
                if re.sub('(.*)\.([a-z])\..*','\\2',str(wordnet.synsets(words[x])[0])) in ['v','adj']

In [ ]:
def preprocessing(text):

    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)

    text = re.sub('\\n',' ',text)

    text = no_abbreviation(text)

    text = re.sub('[\W]+',' ', text.lower())

    text = change_adversatives(text)

    tokens = lemmatizer(text)
    text = ''
    for index, token in enumerate(tokens):

        #tokens[index] = no_typo(token)
        if token in stop:
            tokens[index] = ''
        else:
            text = text + tokens[index] + ' '
    return {'text':text,'emoticons':emoticons}

#### Extract features through LDA in an exact business_ID

In [43]:
print(Counter(train_10000["business_id"]))

Counter({20089: 92, 107629: 86, 13423: 85, 130178: 61, 101708: 47, 143487: 44, 107531: 40, 64416: 38, 142318: 37, 153845: 37, 157910: 33, 118365: 33, 80029: 32, 9439: 30, 192547: 30, 88442: 30, 29046: 29, 13172: 28, 51946: 28, 117426: 26, 150946: 25, 91058: 24, 22814: 22, 149506: 22, 135546: 22, 189440: 21, 41635: 21, 39261: 21, 61023: 20, 111872: 20, 161534: 19, 101022: 19, 76506: 18, 91682: 18, 157645: 18, 35411: 18, 24564: 18, 152949: 18, 142312: 18, 34159: 17, 4836: 17, 61192: 17, 108514: 16, 190583: 16, 38349: 16, 108614: 16, 96069: 15, 67995: 15, 100001: 15, 101458: 15, 153478: 15, 157515: 15, 112712: 15, 185769: 15, 103807: 14, 175746: 14, 51953: 14, 139811: 14, 136289: 14, 145384: 13, 162751: 13, 148427: 13, 1611: 13, 108041: 13, 90950: 13, 105184: 13, 87587: 13, 14301: 13, 18125: 12, 161100: 12, 105242: 12, 175829: 12, 137087: 12, 67059: 12, 100044: 12, 71026: 12, 6939: 12, 72322: 12, 114321: 12, 27860: 12, 152196: 12, 142739: 12, 182464: 12, 92895: 12, 86180: 11, 105381: 11, 

In [57]:
rest1 = train_10000_en.loc[train_10000_en["business_id"] == 20089]

In [61]:
import pandas as pd
import tqdm
from tqdm import tqdm, tqdm_pandas
from multiprocessing import Pool
import multiprocessing
n_cpu = multiprocessing.cpu_count()
import re
import numpy as np
import gensim
from gensim.corpora.dictionary import Dictionary
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
import pyLDAvis.gensim
import pyLDAvis
from gensim.models import CoherenceModel
%matplotlib inline

D:\anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [64]:
from tqdm import tqdm, tqdm_pandas
tqdm.pandas()
dictionary1 = rest1.text.progress_apply(preprocessing)

100%|██████████| 91/91 [00:00<00:00, 169.86it/s]


In [82]:
ggg[1]

['pay',
 'view',
 'strip',
 'disappointed',
 'view',
 'pool',
 'saw',
 'ti',
 'facade',
 'trump',
 'tower',
 'not',
 'worth',
 'extra',
 'price',
 'restaurant',
 'choice',
 'great',
 'food',
 'good',
 'casino',
 'fun',
 'room',
 'okay',
 'would',
 'think',
 'twice',
 'stay',
 'customer',
 'excellant',
 'staff',
 'stand']

In [73]:
text_temp = [dictionary1[i]['text'] for i in rest1.index]

In [74]:
sentence_stream1 = [sent.split(' ') for sent in text_temp]

In [75]:
bigrams = Phraser(Phrases(sentence_stream1, min_count=5, threshold=5))

In [76]:
ggg = bigrams[sentence_stream1]

In [79]:
text_temp_new = []
for i in range(len(ggg)):
    text_temp_new.append(concatenate(ggg[i]))

In [80]:
text_temp_new

['blown away sophisticated beautiful hotel one best la_vega decor breathe take place always smell like magical garden many top notch restaurant spa service show club choose vegas luxury last time stay july 2011 big disappointment total 8 u one bed break fell flat floor foul play 6 8 u leave kind bug bite look like terrible case eczema rash mechanic curtain not_work main v not_work actually disappointed understatement disgust 5star resort hotel fail u miserably please view picture rash',
 'pay view strip disappointed view pool saw ti facade trump tower not worth extra price restaurant choice great food good casino fun room okay would think twice stay customer excellant staff stand',
 'extremely annoy venetian chooses broadcast fox news vip lounge customer employee would consider harassment since neither choice change channel try complain employee state not able change channel trust venetian customer employee not want force watch fox news please change neutral channel cnn provide transpa

In [84]:
import pandas as pd
import tqdm
from tqdm import tqdm, tqdm_pandas
from multiprocessing import Pool
import multiprocessing
n_cpu = multiprocessing.cpu_count()
import re
import numpy as np
import gensim
from gensim.corpora.dictionary import Dictionary
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
import pyLDAvis.gensim
import pyLDAvis
from gensim.models import CoherenceModel
%matplotlib inline

In [85]:
def tokenize(text):
    token = [re.sub(' ','_',x.strip()) for x in text.split(',')]
    token = [x for x in token if x != 'food' and x != '']
    return token

In [86]:
def token(text, cores=n_cpu-1):
    with Pool(processes=cores) as pool:
        result = pool.map(tokenize,text)
    return result

In [87]:
mydict = Dictionary(ggg)

In [88]:
mycorpus = [mydict.doc2bow(text) for text in ggg]

In [91]:
lda_model = LDA.LdaModel(mycorpus,
                         id2word=mydict,
                         num_topics = 5,
                         alpha='auto',
                         per_word_topics=True,
                         random_state = 123)

In [92]:
lda_model.print_topics()

[(0,
  '0.024*"hotel" + 0.023*"room" + 0.016*"stay" + 0.014*"not" + 0.013*"venetian" + 0.010*"but" + 0.008*"time" + 0.008*"great" + 0.008*"suite" + 0.007*"get"'),
 (1,
  '0.018*"room" + 0.018*"not" + 0.015*"stay" + 0.011*"but" + 0.009*"hotel" + 0.008*"great" + 0.007*"casino" + 0.006*"get" + 0.006*"suite" + 0.006*"food"'),
 (2,
  '0.019*"room" + 0.010*"venetian" + 0.009*"not" + 0.007*"but" + 0.007*"hotel" + 0.007*"get" + 0.006*"suite" + 0.006*"need" + 0.005*"stay" + 0.005*"food"'),
 (3,
  '0.015*"stay" + 0.014*"room" + 0.013*"not" + 0.012*"one" + 0.010*"hotel" + 0.009*"but" + 0.008*"get" + 0.007*"venetian" + 0.007*"would" + 0.007*"suite"'),
 (4,
  '0.023*"room" + 0.020*"hotel" + 0.019*"not" + 0.018*"but" + 0.012*"stay" + 0.011*"great" + 0.010*"venetian" + 0.008*"suite" + 0.008*"go" + 0.008*"nice"')]

## Failed, there is nothing useful through this way, maybe because the lack of samples.